In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.data import Dataset, AUTOTUNE
from tensorflow import keras
from keras import losses


2024-10-03 21:09:08.619015: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-03 21:09:08.627374: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-03 21:09:08.636858: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-03 21:09:08.639783: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-03 21:09:08.646750: I tensorflow/core/platform/cpu_feature_guar

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


I0000 00:00:1727978949.597069    2172 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727978949.622085    2172 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727978949.622115    2172 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
raw = open('data2.txt', mode='r', encoding='utf-8').readlines()
data = []
for line in raw:
    if line != '\n' and 'Глава' not in line:
        data.append(' '.join(line.split()[1:]))
    
data = [line.replace('\n', ' ').replace('\xa0', ' ') for line in data]
text = ' '.join(data)

In [4]:
text[:100]

'завет ЗАВЕТ Моисея  В начале сотворил Бог небо и землю. Земля же была безвидна и пуста, и тьма над б'

In [5]:
def get_features_target(seq):
    features = seq[:-1]
    target = seq[1:]
    return features, target

BATCH_SIZE = 128

alphabet = np.array(sorted(set(text)))
sym_to_idx = {}
idx_to_sym = {}

for idx, sym in enumerate(alphabet):
    sym_to_idx[sym] = idx
    idx_to_sym[idx] = sym
    
text_idx = np.array([sym_to_idx[char] for char in text])
sequences = Dataset.from_tensor_slices(text_idx).batch(BATCH_SIZE, drop_remainder=True)

dataset = sequences.map(get_features_target)

data = dataset.batch(BATCH_SIZE, drop_remainder=True).repeat()
data = data.prefetch(AUTOTUNE)

I0000 00:00:1727978950.034695    2172 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727978950.034738    2172 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727978950.034750    2172 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727978950.110099    2172 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727978950.110141    2172 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-03

In [6]:
rnn_model = keras.Sequential([
    keras.layers.Embedding(len(alphabet), BATCH_SIZE),
    keras.layers.SimpleRNN(64, return_sequences=True, stateful=True),
    keras.layers.Dense(len(alphabet))
])

rnn_model.compile(optimizer='adam', loss=losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
rnn_model.fit(data, epochs=40, verbose=1, steps_per_epoch= len(sequences) // BATCH_SIZE)

Epoch 1/40


I0000 00:00:1727978973.694444    2304 service.cc:146] XLA service 0x7f3548009eb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727978973.694475    2304 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4080 SUPER, Compute Capability 8.9
2024-10-03 21:09:33.716457: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-03 21:09:33.780111: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


  6/243 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.0058 - loss: 4.6077

I0000 00:00:1727978975.216927    2304 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


243/243 ━━━━━━━━━━━━━━━━━━━━ 8s 23ms/step - accuracy: 0.2035 - loss: 3.2338
Epoch 2/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.3231 - loss: 2.4023
Epoch 3/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.3615 - loss: 2.2464
Epoch 4/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.3829 - loss: 2.1594
Epoch 5/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.3982 - loss: 2.1027
Epoch 6/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - accuracy: 0.4088 - loss: 2.0616
Epoch 7/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.4167 - loss: 2.0310
Epoch 8/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - accuracy: 0.4218 - loss: 2.0072
Epoch 9/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.4263 - loss: 1.9879
Epoch 10/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.4302 - loss: 1.9719
Epoch 11/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.4336 - loss: 1.9582
Epoch 12/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/ste

In [7]:
def predict_next(sample, model, tokenizer, vocabulary, n_next, rnd_power, batch_size):
    sample_token = [tokenizer[char] for char in sample]
    predicted = sample_token

    sample_tensor = tf.expand_dims(sample_token, 0)
    sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    
    for _ in range(n_next):
        cur = model(sample_tensor)
        cur = cur[0].numpy() / rnd_power
        cur = tf.random.categorical(cur, num_samples=1)[-1, 0].numpy()
        predicted.append(cur)
        sample_tensor = predicted[-99:]
        sample_tensor = tf.expand_dims([cur], 0)
        sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    res = [vocabulary[i] for i in predicted]
    generated = ''.join(res)
    return generated

In [8]:
print(predict_next(
    sample='б',
    model=rnn_model,
    tokenizer=sym_to_idx,
    vocabulary=idx_to_sym,
    n_next=200,
    rnd_power=0.6,
    batch_size=BATCH_SIZE
))

бой место на него законяную, от то ты и своего и вы все рас велевали видел такого по от всем, которые восырноветных народа, и пленаю повелился Иудей моё верение день словам в и стание предых и услышали


In [9]:
print(predict_next(
    sample='1',
    model=rnn_model,
    tokenizer=sym_to_idx,
    vocabulary=idx_to_sym,
    n_next=100,
    rnd_power=0.2,
    batch_size=BATCH_SIZE
))

1 Иерусалим в поставить в вости возвратился и пришёл в сердце в служение на него и возвратили в верно
